In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,32106
2,Huelva,Confirmados PDIA 14 días,1507
3,Huelva,Tasa PDIA 14 días,"293,66486739287177"
4,Huelva,Confirmados PDIA 7 días,690
5,Huelva,Total Confirmados,32286
6,Huelva,Curados,29871
7,Huelva,Fallecidos,386


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  32106.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  8649.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 514 personas en los últimos 7 días 

Un positivo PCR cada 237 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,32106.0,690.0,1507.0,513170.0,134.458367,293.664867,144.0
Huelva-Costa,18487.0,392.0,837.0,289548.0,135.383425,289.071242,84.0
Huelva (capital),8649.0,280.0,607.0,143837.0,194.664794,422.005465,57.0
Condado-Campiña,10430.0,229.0,522.0,156231.0,146.577824,334.120629,50.0
Moguer,1326.0,36.0,82.0,21867.0,164.631637,374.994284,13.0
Aljaraque,1139.0,39.0,81.0,21474.0,181.614976,377.200335,12.0
Sierra de Huelva-Andévalo Central,2839.0,66.0,143.0,67391.0,97.935926,212.194507,10.0
Almonte,1856.0,37.0,104.0,24507.0,150.977272,424.368548,10.0
San Juan del Puerto,532.0,30.0,48.0,9411.0,318.775901,510.041441,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Valverde del Camino,616.0,37.0,77.0,12750.0,290.196078,603.921569,7.0
Villarrasa,179.0,6.0,13.0,2211.0,271.370421,587.969245,0.0
Niebla,240.0,5.0,21.0,4116.0,121.477162,510.204082,1.0
San Juan del Puerto,532.0,30.0,48.0,9411.0,318.775901,510.041441,8.0
Palos de la Frontera,688.0,25.0,55.0,11742.0,212.910918,468.404020,3.0
Almonte,1856.0,37.0,104.0,24507.0,150.977272,424.368548,10.0
Huelva (capital),8649.0,280.0,607.0,143837.0,194.664794,422.005465,57.0
Aljaraque,1139.0,39.0,81.0,21474.0,181.614976,377.200335,12.0
Moguer,1326.0,36.0,82.0,21867.0,164.631637,374.994284,13.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Rociana del Condado,558.0,1.0,14.0,7939.0,12.596045,176.344628,0.0,0.071429
San Bartolomé de la Torre,301.0,2.0,10.0,3761.0,53.177346,265.886732,1.0,0.200000
Gibraleón,636.0,5.0,23.0,12737.0,39.255712,180.576274,2.0,0.217391
Niebla,240.0,5.0,21.0,4116.0,121.477162,510.204082,1.0,0.238095
Isla Cristina,2155.0,5.0,19.0,21393.0,23.372131,88.814098,2.0,0.263158
Cartaya,1356.0,12.0,38.0,20083.0,59.752029,189.214759,2.0,0.315789
Villanueva de los Castillejos,164.0,2.0,6.0,2825.0,70.796460,212.389381,1.0,0.333333
Almonte,1856.0,37.0,104.0,24507.0,150.977272,424.368548,10.0,0.355769
Ayamonte,1322.0,4.0,10.0,21104.0,18.953753,47.384382,-1.0,0.400000
